# Quatrième séance : adressage et mutabilité

Durant cette séance, on étudie la gestion des variables.

## 1) Tout est pointeur !

### a) Les références

En Ada, vous devez déclarer le type d'une variable avant de pouvoir la manipuler.

La déclaration de la variable `Valide` de type booléen permet de réserver l'espace mémoire d'un booléen (un octet). Le nom `Valide` est associé à cet espace jusqu'à la fin de la procédure. La variable `Valide` ne peut changer de type. Si on voulait écrire un Integer dans l'espace associé à `Valide`, l'espace serait insuffisant : un Integer utilise quatre octets.

En Python, vous avez vu que l'on peut modifier le type d'une variable sans problème.

In [ ]:
var=2
var=True
var="ok"

En fait, le variable `var` ne correspond pas au contenu lui-même, mais à l'adresse du contenu. Lorsque l'on écrit `var=True`, plusieurs choses se passent :
- `var` est ajouté à l'espace des variables si elle n'y était pas déjà,
- l'espace mémoire pour un booléen est réservé,
- la valeur `True` est écrite à cet endroit,
- on écrit dans `var` l'adresse de cet espace mémoire.

On dit alors que `var` est une **référence** au booléen. On parle de **référencer** (resp. **déréférencer**) lorsqu'on passe d'un contenu à son adresse (resp. l'inverse).

Mais du coup, pourquoi `print(var)` n'affiche pas l'adresse que contient `var`, mais le booléen ?

In [ ]:
var = True
print(var) #affiche True

Le référencement/déréférencement est géré automatiquement en Python.

In [ ]:
var = 2
var = var + 1

La deuxième ligne cache plusieurs choses :
- l'opérateur `+` va récupérer le contenu pointé par l'adresse contenue dans `var` (et non l'adresse elle-même),
- un nouvel espace mémoire est réservé pour stocker le résultat du calcul.
- l'adresse de ce nouvel espace est écrit dans `var`.

Après l'exécution de la cellule précédente, l'adresse contenu dans `var` est une adresse vers un espace mémoire contenant l'entier $3$.

**Remarque :** On prêtera attention au deuxième point : le contenu pointé par `var` n'est **pas modifié** ! Un nouvel espace contiendra le résultat, même si cela peut vous paraître peu astucieux.

*L'équivalent en ADA :*

### b) L'identifiant d'une variable

Pour voir ce qui se passe, on peut utiliser la fonction `id` qui nous donne l'adresse contenue dans une variable (il n'y a pas de déréférencement cette fois).

In [ ]:
var = 2
print(id(var)) #adresse à laquelle est stockée le 2
var = var + 1
print(id(var)) #nouvelle adresse à laquelle est stockée le 3

Lorsqu'une variable est affectée à une autre, seule l'adresse est copiée, pas le contenu.

In [ ]:
a=1
print(id(a))
b=a
print(id(b))
a=2
print(id(a))
print(id(b))

On voit que `a` et `b` pointent vers une même case contenant `1`. Puis la deuxième affectation de `a` change l'adresse vers laquelle cette variable pointe, mais elle ne modifie pas le contenu de la case contenant `1`. La variable `b` continue donc de pointer vers la valeur `1`.

**Exercice :** à l'aide de la fonction `id`, comprenez comment est géré le passage d'une variable en paramètre d'une fonction. S'agit-il d'un passage par valeur (copie de la valeur) ou par référence (copie de l'adresse) ?

In [ ]:
# code à compléter avec des affichages

def mul(a):
    a = 2*a

def add(a):
    a=a+0 
    mul(a)
    a=a+1
    return a
    

a = 2023
b = add(a)

### c) Valeurs internes

Vous avez peut-être remarqué quelque chose de suspect. Dans les deux cellules précédentes, le `2` est stocké au même endroit alors qu'on a dit qu'un nouvel espace était réservé à chaque fois. Il y a certains cas où ce n'est pas vrai :
- Certaines valeurs standards sont dits internes et ne seront écrites qu'à un seul endroit (petits entiers, booléens, petits strings sans caractères spéciaux). Si deux variables référencent une de ces valeurs, pas la peine de réserver deux espaces, la deuxième pointera là où pointe la première.
- D'autres types permettent d'éviter cela (partie 3 de ce TD).

In [ ]:
a = 18
print(id(a))
a = 18
print(id(a))
print(id(18))

a = 8439439903930
print(id(a))
a = 8439439903930
print(id(a))
print(8439439903930)

**Exercice :** écrire un petit programme qui permet de calculer le plus petit entier naturel qui n'est pas déjà stocké en mémoire.

In [ ]:
# à compléter

## 2) Une valeur est toujours bien accompagnée

### a) Par un gars...

Vous vous souvenez qu'en Python, le typage est dynamique. On pourrait imaginer que chaque variable stocke avec elle le type du contenu vers lequel elle pointe et que ce dernier est modifiable.

In [ ]:
a=2
print(type(a)) #integer
a="ok"
print(type(a)) #string

En Python, c'est en fait le contenu lui-même qui stocke son type. Une variable ne pointe pas juste vers une valeur mais un type article contenant (au moins) une valeur et un type. On rappelle que mémoriser le type est primordial pour savoir comment lire la mémoire contenant la valeur.

### b) ... et par son nombre de matchs

Un troisième attribut est toujours stocké avec une valeur et son type. Pour comprendre son intérêt, prenons l'exemple ci-dessous en ADA.

Si on alloue de la mémoire dynamiquement (avec `new`) sans jamais libérer celle qui n'est plus utilisée, on a ce qu'on appelle une fuite de mémoire. L'espace mémoire utilisé durant la boucle n'est plus accessible (on a perdu les adresses), mais le système d'exploitation n'a aucune raison de deviner qu'il peut le récupérer pour le redistribuer. On rajoute donc une libération de mémoire.

La bonne nouvelle, c'est qu'en Python, vous n'aurez pas à gérer la libération de la mémoire. Mais alors, comment ça marche ?

Une variable pointe vers une troisième information : un compteur de référence. Ce compteur, comme on s'en doute, permet de mémoriser combien de variables pointent directement ou indirectement (via une collection par exemple) vers le contenu associé. On peut récupérer ce compteur avec la fonction `sys.getrefcount`.

**Remarque :** la fonction `sys.getrefcount` peut renvoyer une valeur supérieure au nombre de références auxquelles on pense : <a href="https://docs.python.org/3/library/sys.html#sys.getrefcount">lien vers la doc.</a> En particulier, la fonction elle-même peut en créer.

In [ ]:
#réinitialise l'espace des variables
%reset -f 

import sys #pour pouvoir utiliser getrefcount

potiron = "potiron"
print("nb ref du contenu pointé par potiron au début= ",sys.getrefcount(potiron))
legume=potiron
tarte=potiron
soupe=potiron
print("nb ref du contenu pointé par potiron au début= ",sys.getrefcount(potiron))

Le compteur de référence est incrémenté de trois après avoir affecté trois variables à son contenu.

De même, quand on affecte une variable par une autre, le nombre de référence augmente. Remarquez avec l'exemple suivant qu'une définition avec une valeur donnée explicitement peut créer un nouvel espace.

In [ ]:
%reset -f 
import sys

print("nb ref du contenu 2023 = ", sys.getrefcount(2023))
a=2023
print("nb ref du contenu pointé par a au début = ",sys.getrefcount(a))
b=a
c=2023 #nouvel espace mémoire
print("nb ref du contenu pointé par a à la fin = ",sys.getrefcount(a))
print("nb ref du contenu pointé par b à la fin = ",sys.getrefcount(b))
print("nb ref du contenu pointé par c à la fin = ",sys.getrefcount(c))

Ce compteur de référence permet de libérer la mémoire intelligemment : si il tombe à $0$, c'est que rien ne l'utilise. On peut donc libérer l'espace associé. Imaginez donc un `free` automatique. Attention que des fuites de mémoire sont encore possibles...

**Résumé :**

Une variable pointe donc (au moins) vers :
- type, 
- un compteur de référence,
- une valeur.

*Ce triplet peut-être vu comme un record en ADA. On ne peut pas directement accéder aux attributs à l'instar des record privé. On ne peut accéder qu'à certains indirectement via des méthodes.*

### c) Espace utilisé

On peut voir l'espace mémoire réservé grâce à la fonction `sys.getsizeof` (l'unité est l'octet) :

In [ ]:
a=3.14893303
print(sys.getsizeof(a))
print(a.__sizeof__()) #quasi équivalent

A priori, sur une machine 64-bit, les flottants sont représentés sur 24 octets :
- huit octets (64 bits) pour le type, 
- huit pour le compteur de référence,
- huit pour la valeur.

Si vous avez la curiosité de tester sur d'autres types, vous serez surpris. Par exemple, les entiers sont de taille variable : $0$ ne prend que $24$ octets, alors que $1$ en prend $28$. 

Un entier contient lui aussi un type, un compteur de référence et une valeur. Mais cette valeur étant de taille variable, il stocke aussi cette taille sur $8$ octets. Par exemple :
- Pour $0$, la taille vaut $0$ (sur $8$ octets) et il n'y a pas de valeur ($0$ octets). Avec le type et le refcount, on a bien $24$ bits.
- pour $1$, la taille vaut $4$ et la valeur est $000...001$ ($32$ bits car $4$ octets). Au total, on a bien $28$ bits.

De même, `False` et `True` ne prennent pas le même espace : `False` est vu comme $0$, `True` comme $1$.

In [ ]:
print(sys.getsizeof(0))
print(sys.getsizeof(1))
print(sys.getsizeof(2**30-1))
print(sys.getsizeof(2**30))


print(sys.getsizeof(False))
print(sys.getsizeof(True))

### d) Une valeur spéciale : None

Vous retrouver en Python l'équivalent du pointeur `null` du langage ADA. C'est `None`.

In [ ]:
import sys

print(id(None))
print(type(None))
print(sys.getrefcount(None))
print(sys.getsizeof(None)) #le type et le refcount

Le `None` est très pratique pour les fonctions qui doivent renvoyer un résultat particulier dans certains cas. Par exemple, si vous chercher le premier indice d'une valeur dans une liste ne la contenant pas, vous pouvez utiliser des exceptions, mais aussi renvoyer `None` :

In [ ]:
def chercher(l,x):
    for i in range(len(l)):
        if l[i]==x:
            return i
    return None #l'élement x n'est pas dans la liste l

print(chercher([1,3],2))

**Remarque :** ne rien renvoyer est équivalent à renvoyer `None`. Une fonction qui ne renvoie rien dans tous les cas est l'équivalent d'une procédure en ADA.

### e) Collections : toujours plus de pointeurs

Les collections fonctionnent de façon similaire. On va regarder le cas des listes.

In [ ]:
l=[1,2,3]
m=[1,2,3]
n=m
print("id(l)=",id(l))
print("id(m)=",id(m))
print("id(n)=",id(n))
print("l==m : ", l==m)
print("l is m :", l is m)
print("n==m : ", n==m)
print("n is m : ", n is m)

On voit que deux listes contenant les mêmes éléments mais définies indépendemment sont à différents endroits en mémoire. En revanche, affecter une liste avec une autre ne copie que le pointeur et non le contenu. Il faut donc être vigilant quand on modifie une liste.

In [ ]:
l=[1,2,3]
m=l
l.append(4)
print(m)

Pour copier une collection, utilisez la méthode `copy` (attention, une collection peut en cacher une autre) :

In [ ]:
l=[1,2,3]
m = l.copy()
l.append(4)
print(m)

Pour l'organisation du contenu pointé, c'est plus compliqué. Une liste contient (entre autre) :
- un type,
- un compteur de référence,
- une capacité : nombre de cases réservées (cf TD2),
- une taille : nombre de cases réellement utilisées,
- un pointeur vers un tableau.

Ce pointeur de tableau est important. Dans l'exemple suivant, si toute la capacité de `l` est utilisée et que pour ajouter un élément, il faut copier le contenu ailleurs, on veut que toutes les références à la liste soient toujours valides. 

In [ ]:
l = [1,2,3,4]
m=l
l.append(5)
print(m) #m doit toujours pointer sur la même liste que l

L'intérêt majeur d'un tableau est de pouvoir accéder à n'importe quelle case en temps constant. Pour accéder à la $i$-ème case d'un tableau, il suffit de rajouter à l'adresse du début du tableau la taille d'une case multipliée par $i$. 

*Par exemple, si la taille d'un entier est $4$ octets, pour accéder à la $10$-ème case d'un tableau d'entiers, il suffit de prendre l'adresse du début du tableau et d'y ajouter $40$.*

Le dernier attribut (pointeur vers tableau) du type `list` décrit ci-dessus est donc un pointeur de pointeur : l'adresse de la case qui stocke l'adresse du début du tableau.

Depuis le TD2, vous savez que le type `list` stocke les éléments avec un tableau. Le problème est que les éléments peuvent être de tailles différentes. On a vu par exemple que les entiers sont de taille variable. La solution adoptée est d'avoir un tableau de pointeurs (vers les valeurs) et non de valeurs. Les pointeurs sont toujours de même taille.

**Exercice :**

1) A l'aide de la fonction `sys.getsizeof`, comment récupérer la capacité (nombre de cases disponibles) d'une variable de type `list` ?

In [ ]:
# à compléter

2) Et le nombre de cases réellement utilisées ?

In [ ]:
# à compléter

3) La capacité est-elle vraiment doublée à chaque fois qu'elle est complètement utilisée ?

In [ ]:
# à compléter

4) Dessiner sur papier les différentes cases mémoire utilisées par les variables définies ci-dessous, et les liens de référence entre elles. Si $p_a$ est un pointeur sur $a$, on fait une flèche de $p_a$ vers $a$.

In [ ]:
l=[1,True,2.4]
m=l

## 3) Mutabilité

On a déjà vu que certaines collections sont modifiables et d'autres non. Ce caractère s'appelle la mutabilité. Un type est mutable si on peut modifier le contenu d'une variable de ce type, non mutable sinon.

### a) Les types non mutables

Les types de bases que vous manipulez couramment sont tous non mutables : `int`, `float` et `bool`. En effet, on a vu qu'incrémenter un entier modifie son `id` : il est réaffecté et non modifié.

Les collections non mutables que vous avez vu pour l'instant sont les suivantes :
- `tuple`, 
- `str`.

Les intérêts de ces types sont :
- de ne pas risquer de les modifier par erreur (sécurité),
- de pouvoir les utiliser en clé d'un dictionnaire (nécessaire au bon fonctionnement des tables de hachages sous jacentes),
- d'être a priori plus léger : par exemple, les tuples n'ont pas besoin de stocker un attribut de capacité, nécessaire aux listes.

Lorsqu'un paramètre de type non mutable est donné à une fonction, il n'y aura pas d'*effet de bord* : toute modification interne à la fonction ne sera pas répercutée à l'extérieur de celle-ci.

**Questions :** La modification d'une chaîne de caractère peut demander de tout déplacer. On utilisera des caractères spéciaux pour que les chaînes ne soient pas internes (! et ? feront très bien l'affaire).

a) Etudier si ce déplacement est réellement fait en pratique, en ajoutant des caractères un à un à la fin d'une chaîne intialement vide. 

In [ ]:
# à compléter

b) Et si on modifie le contenu d'une chaîne non vide ?

In [ ]:
# à compléter

### b) Les types mutables

Les types mutables que vous avez vu pour l'instant :
- `list`, 
- `dict`.

A l'inverse, ces types sont importants lorsqu'on a besoin d'une structure dynamique. 

**Attention :** ces types introduisent des effects de bords. Une modification interne à une fonction peut avoir un impact à l'extérieur. De même, si deux variables pointent vers le même contenu mutable, modifier le contenu de l'une d'entre elles le modifie pour l'autre (on l'a vu dans la partie précédente).

In [ ]:
l = [1,0,9,2,7,4,3,8,2,1]

In [ ]:
def transforme(l):
    for i in range(len(l)):
        if l[i]%2==0:
            l[i]=0

In [ ]:
transforme(l)
print(l)

La fonction `transforme` met tous les entiers pairs à $0$. Si on s'est trompé et qu'on voulait ne le faire que pour les nombres impairs, il faut faire attention à redéfinir la liste `l` initiale (ce qui peut être bien plus lourd parfois). Essayez de ne modifier que la définition de `transforme` et de relancer le test.

**Remarque :** on peut convertir une collection d'un type vers un autre pour profiter de son caractère mutable ou non mutable. Il suffit d'utiliser la fonction ayant pour nom le type cible (techniquement, on appelle ça un constructeur). 

Dans l'exemple suivant, on veut appliquer une permutation aléatoire à une chaîne de caractères. Pour cela, choisit deux indices aléatoire $r1$ et $r2$ et on transpose les deux caractères à ces indices. Et on itère ce procédé. Afin d'éviter de copier toute la chaîne à chaque transposition, on convertit en amont la chaîne de caractères en liste.

In [ ]:
def suivant(u):
    """
    renvoie un entier pseudo aléatoire
    """
    return (227608 * u + 9204) % 59049

def suivant_couple(u):
    """
    renvoie un couple d'entiers pseudo aléatoire
    """
    return suivant(u),suivant(suivant(u)) 

def permut(mot):
    """
    applique une série de transposition entre des couples d'indices pseudo aléatoires
    """
    u2 = 3
    for i in range(10000):
        u1,u2 = suivant_couple(u2) #choix d'une paire d'indice aléatoire
        r1,r2 = u1%len(mot),u2%len(mot)
        tmp = mot[r1]
        mot[r1] = mot[r2]
        mot[r2] = tmp


mot = "Win the yes needs the no to win against the no"
mot_list = list(mot) #on convertit en amont
permut(mot_list)
resultat = "".join(mot_list) #conversion inverse, join assemble les éléments de la liste avec "" en séparateur
print(resultat)

Vous découvrez au passage comment générer facilement une suite $(u_n)_n$ pseudo aléatoire d'entiers entre $0$ et $m-1$ :
- $u_0$ fixé (la graine, à modifier pour un tirage différent),
- $u_{n+1} = (a\cdot u_n + b) \mod c$,

avec certaines bonnes propriétés pour $a,b,c$ :
- $a-1$ multiple de tout facteur premier (et $4$) de $c$, 
- $b$ et $c$ premiers entre eux.

## 4) Exercices 

**Exercice 1 :** dessiner sur papier les différentes cases utilisées par la variable `l` définie ci-dessous, et les liens de référence entre elles.

In [ ]:
l = [[0,1],True,([False,[True,2],3],False,2),False]

**Exercice 2 :** on veut implémenter une liste chaînée avec les outils qu'on a pour l'instant. On rappelle qu'une liste chaînée permet d'accéder à l'élément du début et à la sous liste privée du premier élément.

1) Vaut-il mieux utiliser des couples ou des listes à deux éléments ? `(element,suivant)` ou `[element,suivant]`

2) Implémenter une fonction d'insertion à la fin d'une liste chaînée.

In [ ]:
# à compléter

**Exercice 3 :** graphes d'intervalles.

On doit affecter des salles à un ensemble de cours. Chaque cours se déroule sur un intervalle de temps, fixé à l'avance. Lorsqu'on parlera de cours, on sous entendra l'intervalle correspondant. On ne manipulera que des intervalles ouverts, i.e. bornes exclues.

i) Comment représenter en mémoire un cours ?

ii) On dit que deux cours sont en conflits si il existe un même instant $t$ pendant lequel les deux cours se déroulent. Traduire mathématiquement cette propriété, puis implémenter une fonction `conflit` qui :
- prend en paramètre deux cours, 
- renvoie si ils sont en conflits ou non .

In [ ]:
# à compléter

In [ ]:
# tests à compléter

iii) On se donne une liste `cours` de cours. On veut créer une structure associant à chaque cours l'ensemble des cours en conflits avec lui. C'est ce qu'on appelle un graphe d'intervalles. Quelle structure utiliser ? Implémenter une fonction `graphe_intervalle` qui prend en entrée une liste de cours et renvoie le graphe d'intervalles associé. **On supposera dans toute la suite qu'il n'y a pas de doublon, i.e. pas deux cours avec exactement le même intervalle.**

In [ ]:
# à compléter

In [ ]:
# tests à compléter

iv) Pour les intervalles $I_1=]1,3[,\ I_2=]2,5[$ et $I_3=]2,4[$, dessiner sur papier les cases mémoires utilisées par le graphe d'intervalles, et leur liens.

v) Les salles sont des chaînes de caractères : "SS12", "GEI024", "GM116", ... Ecrire une fonction `min_dispo` qui :
- prend en entrée une liste `salles` des salles existantes et une liste `utilise` de salles utilisées,
- renvoie la salle d'indice minimal (dans `salles`) qui n'est pas dans `utilise`. 

Quelle optimisation peut-on facilement faire sur `utilise` ?

In [ ]:
# à compléter

In [ ]:
# tests à compléter

vi) On va progressivement associer une salle à chaque cours. Quelle structure utiliser pour stocker ces association ? Il faut prendre en compte le fait qu'au début, un cours n'a pas de salle.

In [ ]:
# à compléter

vii) On considére les cours dans l'ordre de leur heure de début (et de fin si égalité). Comment s'appelle cet ordre ? *(non ce n'est pas l'ordre alphabétique)*. L'ordre initial des intervalles est quelconque. Faire en sorte qu'on puisse parcourir les intervalles dans l'ordre voulu.

In [ ]:
# à compléter

viii) Ecrire une fonction `indispo` qui :
- prend en entrée un ensemble de cours et une structure qui associe à chaque cours une salle (question vi),
- renvoie l'ensemble des salles utilisées par ces cours.

In [ ]:
# à compléter

In [ ]:
# tests à compléter

ix) Ecrire une fonction `affectation` qui :
- prend en entrée une liste de cours et une liste de salles,
- renvoie une affectation de salle pour chaque cours. 

Les cours sont parcourus dans l'ordre de la question vii). Pour chaque cours, on lui affecte la salle d'indice minimale qui n'est pas utilisée par les cours en conflits avec lui.

In [ ]:
# à compléter

In [ ]:
# tests à compléter

x) Quelle est la complexité de l'algorithme utilisé ?

xi) Le nombre de salle utilisé est-il le minimum ?